In [1]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.
/Users/ambroiserenaud/anaconda3/envs/chili/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ambroiserenaud/anaconda3/envs/chili/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ambroiserenaud/anaconda3/envs/chili/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [2]:
data = pd.read_csv('train.csv').iloc[:, :]
vocab = ['C_E_F_T',
         'C_E_F_C',
         'C_E_F_O',
         'A_E_F_T',
         'A_E_F_O',
         'A_E_F_C',
         'G_E_F_C',
         'G_E_F_T',
         'G_E_F_O',
         'A_E_M_T',
         'A_E_M_O',
         'A_E_M_C',
         'G_E_M_O',
         'G_E_M_C',
         'G_E_M_T',
         'C_E_M_O',
         'C_E_M_C',
         'C_E_M_T',
         'C_H_F_CO',
         'C_H_F_CT',
         'C_H_F_OT',
         'G_H_F_OT',
         'G_H_F_CO',
         'G_H_F_CT',
         'A_H_F_CT',
         'A_H_F_OT',
         'A_H_F_CO',
         'C_H_M_CO',
         'C_H_M_CT',
         'C_H_M_OT',
         'A_H_M_CT',
         'A_H_M_OT',
         'A_H_M_CO',
         'G_H_M_OT',
         'G_H_M_CO',
         'G_H_M_CT', ]

labels = ['correct',
          'wrong',
          'type',
          'orientation',
          'color']

types = ['INTRO',
         'CORE',
         'FLEX',
         'TRIK',
         'DELY'
]

topics = ['cards',
          'animals',
          'geometry'
    
]

feat = ['type',
        'color',
        'orientation',
        'dual'
]

age = ['8-10','11-13']

In [3]:
def seq_to_int(qts,vocab, labels, types, feat, topics, n_steps, age):
    integ = list()
    for i,x in enumerate(qts):
        if i != n_steps:
            features = list()
            #features.append(vocab.index(qts[i, 4]))
            #features.append(qts[i, 2])
            features.append(types.index(qts[i, 9]))
            features.append(feat.index(qts[i, 10]))
            features.append(topics.index(qts[i, 6]))
            features.append(age.index(qts[i, 7]))
            features.append(labels.index(qts[i, 5]))
        else:
            features = list()
            #features.append(vocab.index(qts[i, 4]))
            #features.append(qts[i, 2])
            features.append(types.index(qts[i, 9]))
            features.append(feat.index(qts[i, 10]))
            features.append(topics.index(qts[i, 6]))
            features.append(age.index(qts[i, 7]))
            features.append(-1)
        integ.append(features)
    return integ

def split_sequence(data, n_steps, vocab, labels, types, feat, topics, age):
    X, Y = list(), list()
    users = list(dict.fromkeys(data.loc[:, "user"]))
    for u in users:
        sequence = data[data.user == u]
        for i in range(len(sequence)):
            end_idx = i + n_steps
            if end_idx > len(sequence)-1:
                break
            x = seq_to_int(sequence.iloc[i:end_idx+1, :].values, vocab, labels, types, feat, topics, n_steps, age)
            y = labels.index(str(sequence.iloc[end_idx, 5]))
            X.append(x)
            Y.append(y)
    return np.array(X), np.array(Y)

In [19]:
data_train = data.iloc[:]
y_train = list()
X_train = list()


for i in tqdm(range(10,18)):
    X_seq, y_seq = split_sequence(data_train, i, vocab, labels, types, feat, topics, age)
    for x in X_seq:
        X_train.append(x)
    for _y in y_seq:
        y_train.append(_y)
#y_train = np_utils.to_categorical(y_train)
y_train = np.asarray(y_train)
X_train = np.asarray(X_train)

100%|██████████| 8/8 [00:29<00:00,  3.69s/it]


In [20]:
data_val = pd.read_csv('validation.csv')
y_val = list()
X_val = list()


for i in tqdm(range(10,18)):
    X_seq, y_seq = split_sequence(data_val, i, vocab, labels, types, feat, topics, age)
    for x in X_seq:
        X_val.append(x)
    for _y in y_seq:
        y_val.append(_y)
#y_train = np_utils.to_categorical(y_train)
y_val = np.asarray(y_val)
X_val = np.asarray(X_val)
X_val = pad_sequences(X_val, value=99)

100%|██████████| 8/8 [00:03<00:00,  2.60it/s]


In [21]:
X_train = pad_sequences(X_train, value=99)

In [22]:
n_hidden = int(2/3 * (X_train.shape[1]+X_train.shape[2]))

In [23]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=5))
amsgrad = Adam(amsgrad=False)
model.add(Activation('softmax'))
model.compile(optimizer=amsgrad, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 18, 50)            11200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 18, 50)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 255       
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
Total params: 31,655
Trainable params: 31,655
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class_weight = {
    0: 1.,
    1: 1.,
    2: 5.,
    3: 5.,
    4: 5.
}
history = model.fit(X_train, np.asarray(y_train), epochs=500, batch_size=300, shuffle=True, verbose=1, validation_data=(X_val, y_val), class_weight=class_weight)

Train on 59764 samples, validate on 5940 samples
Epoch 1/500
59764/59764 [==============================] - 48s 810us/step - loss: 5.2251 - sparse_categorical_accuracy: 0.3754 - val_loss: 1.0871 - val_sparse_categorical_accuracy: 0.4559
Epoch 2/500
59764/59764 [==============================] - 50s 832us/step - loss: 4.9360 - sparse_categorical_accuracy: 0.4648 - val_loss: 0.9901 - val_sparse_categorical_accuracy: 0.5022
Epoch 3/500
59764/59764 [==============================] - 52s 862us/step - loss: 4.6284 - sparse_categorical_accuracy: 0.5194 - val_loss: 0.9131 - val_sparse_categorical_accuracy: 0.5503
Epoch 4/500
59764/59764 [==============================] - 46s 767us/step - loss: 4.2675 - sparse_categorical_accuracy: 0.5609 - val_loss: 0.8385 - val_sparse_categorical_accuracy: 0.5774
Epoch 5/500
59764/59764 [==============================] - 46s 769us/step - loss: 3.9406 - sparse_categorical_accuracy: 0.5865 - val_loss: 0.7849 - val_sparse_categorical_accuracy: 0.5998
Epoch 6/500

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

In [ ]:
#model.save("model2.h5")
#from keras.models import load_model
#model = load_model("model2.h5")

### Test

In [ ]:
data_test = pd.read_csv('test.csv')

In [ ]:
y_test = list()
X_test = list()


for i in tqdm(range(10, 18)):
    X_seq, y_seq = split_sequence(data_test, i, vocab, labels, types, feat, topics, age)
    for x in X_seq:
        X_test.append(x)
    for _y in y_seq:
        y_test.append(_y)
#y_test = np_utils.to_categorical(y_test)
y_test = np.asarray(y_test)
X_test = np.asarray(X_test)
X_test = pad_sequences(X_test, value=99)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred.argmax(axis=1)))
matrix = confusion_matrix(y_test, y_pred.argmax(axis=1))
print(matrix)